In [1]:
import os
import pandas as pd
import boto3
from io import BytesIO
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role
import glob2
import numpy as np

In [2]:
'''
-----------------------------------------------------------------------------
--- Inputs
-----------------------------------------------------------------------------
'''
dataColumn = 'L6'
coor = 'Char'

resultsList = ['Indicies','Response']
#interpolationList = ['NoInterp','Interp']
analysisList = ['AE_Simple','DEC_Simple','PCA']
interpolationList = ['NoInterp']
#analysisList = ['KMA']

In [3]:
'''
-----------------------------------------------------------------------------
--- Definitions
-----------------------------------------------------------------------------
'''

# Configuring S3
s3_bucket_name = 'jasper-ml-sagemaker'
role = get_execution_role()

client = boto3.client('s3')
resource = boto3.resource('s3')
my_bucket = resource.Bucket(s3_bucket_name)

In [35]:
'''
-----------------------------------------------------------------------------
--- Transfer Results
-----------------------------------------------------------------------------
'''

for interpolation in interpolationList:
    for analysis in analysisList:
        
        resultsPath = '{}/{}/{}/'.format(results, interpolation,analysis)

        files = []
        for object_summary in my_bucket.objects.filter(Prefix=resultsPath):
            files.append(object_summary.key[len(resultsPath):])
            
        dimensionList = []
        for file in files:
            if file != '.DS_Store':
                dimensionList.append(file.split('/')[0])
        
        dimensionList = list(set(dimensionList))
        
        for dimension in dimensionList:
            resultsPath = '{}/{}/{}/{}/'.format(results, interpolation,analysis,dimension)
            newResultsPath = '{}/{}/{}/{}/{}/{}/'.format(dataColumn,coor,interpolation,analysis,dimension,results)            
            
            files = []
            for object_summary in my_bucket.objects.filter(Prefix=resultsPath):
                if object_summary.key[len(resultsPath):] != '.DS_Store':
                    files.append(object_summary.key[len(resultsPath):])
                    
            for dataKey in files:
                obj = client.get_object(Bucket=s3_bucket_name, Key=resultsPath + dataKey)
                pd.read_csv(obj['Body'],header = None).to_csv(dataKey, index=False)
                my_bucket.upload_file(dataKey,Key=newResultsPath + dataKey)
                os.remove(dataKey)

In [50]:
'''
-----------------------------------------------------------------------------
--- Transfer Indicies and Responses
-----------------------------------------------------------------------------
'''
for results in resultsList:
    for interpolation in interpolationList:
        for analysis in analysisList:

            resultsPath = '{}/{}/{}/'.format(results, interpolation, analysis)
            newResultsPath = '{}/{}/{}/{}/{}/'.format(dataColumn,coor,interpolation,analysis,results)            

            files = []
            for object_summary in my_bucket.objects.filter(Prefix=resultsPath):
                if object_summary.key[len(resultsPath):] != '.DS_Store':
                    files.append(object_summary.key[len(resultsPath):])
            
            for dataKey in files:
                
                obj = client.get_object(Bucket=s3_bucket_name, Key=resultsPath + dataKey)
                np.save(dataKey,np.load(BytesIO(obj['Body'].read())))
                my_bucket.upload_file(dataKey,Key=newResultsPath + dataKey)
                os.remove(dataKey)